In [1]:
import os
import pandas as pd
import pdfplumber
from langchain_community.chat_models import AzureChatOpenAI
from azure.core.credentials import AzureKeyCredential
import mlflow
from dotenv import load_dotenv
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets


In [2]:
pd.set_option("display.max_colwidth", None)

In [3]:
#Using dotenv in order to access my secret keys, could be to the LLM, vector database, search etc. It keeps it connected to Azure
dotenv_path = '/.env'
load_dotenv(dotenv_path=dotenv_path)


True

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text_content = [page.extract_text() for page in pdf.pages if page.extract_text()]
    return "\n".join(text_content)

In [6]:
# Path to your PDF file
pdf_path = "ESG Knowledge Library\Annual-Report-2023.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

# Creating a DataFrame including the source
data = {
    'text': [pdf_text],  # Assuming pdf_text contains the full text from your PDF
    'source': [pdf_path]  # Include the PDF file name as source
}

In [7]:
from datasets import Dataset
import pandas as pd

In [8]:
df = pd.DataFrame(data)

In [9]:
# Convert DataFrame to Dataset
ds = Dataset.from_pandas(df)


In [10]:
# Example operations on the dataset
ds = ds.map(lambda example: {'length': len(example['text'])})  # Adding a column with the length of each text


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

{'text': 'snoitulos\nruo\nerolpxE\nmoc.ssofnad.www\nStrong partnerships\nand bold investments\nAnnual Report 2023\nDanfoss Annual Report 2023 Letter from the CEO Danfoss at a glance Our purpose Our business Our strategy Sustainability ESG statements Financial statements 2 /158\nPAGE 3-78 PAGE 79-158\nManagement’s Review Financial Statements\nLetter from the CEO Sustainability Financial review\n3 Letter from the CEO 32 Driving sustainability at Danfoss 80 Financial review\n34 Materiality of sustainability topics\nDanfoss at a glance Group accounts and notes\n6 Danfoss in brief Environmental 84 Group accounts\n7 Our transformation highlights 35 Decarbonization 88 Group notes\n13 In 2023, we continued our positive development 41 Circularity 130 Group companies\n14 Financial highlights 43 Environmental performance\n15 Outlook 2024 44 Product safety and compliance Parent accounts and notes\n133 Management’s review for Danfoss A/S\nOur purpose Social 134 Parent accounts\n17 Global megatrends

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument


In [12]:
langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

In [14]:
def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [15]:
from huggingface_hub import InferenceClient
from langchain.llms.base import LLM
from typing import List
import json
from langchain.llms.base import BaseLLM


In [16]:
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.retrievers import AzureAISearchRetriever


In [17]:
# Initialize the Hugging Face Inference Client
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm_client = InferenceClient(model=repo_id, timeout=120)

In [18]:
class HuggingFaceInferenceLLM(LLM):
    def __init__(self, model_id: str, timeout: int = 120):
        self.inference_client = InferenceClient(model=model_id, timeout=timeout)

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        response = self.inference_client.post(
            json={
                "inputs": prompt,
                "parameters": {"max_new_tokens": 1000},
                "task": "text-generation",
            },
        )
        return json.loads(response.content.decode())[0]["generated_text"]

    @property
    def _llm_type(self) -> str:
        return "huggingface-inference"

    def _generate(self, prompts: List[str], stop: List[str] = None) -> List[str]:
        responses = []
        for prompt in prompts:
            response = self._call(prompt, stop)
            responses.append(response)
        return responses


In [19]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [20]:
import random

In [22]:
N_GENERATIONS = 5  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 5 QA couples...


  0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer,source_doc
0,"The decarbonization levers for scope 3 are complex and performance improvements, rolled out to key that at least half of all aluminum used\nand often require innovation and partnerships suppliers in the coming years. is composed of secondary aluminum\nall along the value chain. We embrace this as an by 2030.\nopportunity to collaborate with our business Downstream emissions:\nIt was important to us at Danfoss to\npartners, suppliers, peers, and customers to achieve Decarbonizing with our customers\nhave a clear path towards delivering on\nour shared goals of industry decarbonization. Our downstream scope 3 emissions from the use our First Movers Coalition commitment. About the First Movers Coalition\nof sold products account for around 96% of our total\nBy the time we announced our ∙ 90+ leading global companies\nUpstream emissions: carbon footprint and amount to approximately\ncommitment, we had begun ∙ A partnership by the US State Department and\nDecarbonizing with our suppliers 122 million tons annually. negotiations with suppliers and had World Economic Forum\nUpstream emissions make up around 4% of our secured the first contracts for low- ∙ Represents EUR 14 billion in annual demand, the world’s\ntotal carbon footprint. This is where we have In the segment decarbonization roadmaps, carbon aluminum. largest demand signal\ninfluence through our partnership with suppliers. we identified levers, opportunities, and also ∙ The seven hard-to-abate sectors covered by the FMC account\nIn the upstream decarbonization roadmaps roadblocks and learnings. Highlights of levers for 30% of total global emissions\ndeveloped in 2023, we identified key levers that and opportunities include: further investment\nwill guide our work in the coming years: in optimization and energy efficiency, transition\nto next-generation technologies, and automation.",What percentage of Danfoss's total carbon footprint is made up of upstream emissions?\n,Upstream emissions make up around 4% of Danfoss's total carbon footprint.,ESG Knowledge Library\Annual-Report-2023.pdf


The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in this paper:

Groundedness: can the question be answered from the given context?
Relevance: is the question relevant to users? For instance, "What is the date when transformers 4.29.1 was released?" is not relevant for ML practicioners.
One last failure case we’ve noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like "What is the name of the function used in this guide?". We also build a critique agent for this criteria:

Stand-alone: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be What is the function used in this article? for a question generated from a specific blog article.
We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.